The crawler works back till 2008, there it should raise an 

In [1]:
import requests
import bs4

import pandas as pd
import numpy as np

In [2]:
class UnsupportedPageFromat(RuntimeError):
  pass

old_header_reference = ['Név/konfiguráció',
 'A krém egységessége és tartóssága',
 'íz harmónia (Az édes, a savas és a keserű egyensúlya)',
 'Testesség (testes, lekerekedett, sima)',
 'Ár/érték arány',
 'Összpontszám']

new_headers = ['\nNév', '\nKrém', '\nÍz', '\nTest', '\nÁr', '\nÖssz:']

def is_compatible_with_old_header(headers):
  return all((
    'Név' in headers[0],
    'krém' in headers[1],
    'íz' in headers[2],
    'Testesség' in headers[3],
    'Ár/érték' in headers[4],
    'Össz' in headers[5]
  ))
  

def scrap(table):
  itr = iter(table.find_all('tr'))
  headers = [row.text for row in next(itr).find_all('td')]
  rows = []
  while True:
    try:
      row = next(itr)
      row = [row.text for row in row.find_all('td')]
      rows.append(row)
    except StopIteration:
      break
  return headers, rows

def mine_scores(page):
  tables = page.find_all('table')
  if len(tables) == 1: # new page format
    return scrap(tables[0])
  else:  # old page format
    headers, rows = scrap(tables[1])
    if is_compatible_with_old_header(headers):
      return new_headers, rows
    else:
      raise RuntimeError

def mine_test_page(page_url):
  resp = requests.get(f'{page_url}')
  if resp.status_code != 200:
    raise RuntimeError
  
  test_page = bs4.BeautifulSoup(resp.text)
  try:
    headers, rows = mine_scores(test_page)
  except Exception as e:
    raise UnsupportedPageFromat from e 
  return {
    'headers' : headers,
    'ratings' : rows,
    'soup': test_page,
    'url': page_url,
  }

In [3]:
results = []
host = 'https://www.kavekorzo.hu'
i=0

while True: 
  gallery_url = f'/index.php?option=com_content&view=category&layout=blog&id=43&Itemid=110&limitstart={i}'
  
  resp = requests.get(f"{host}{gallery_url}")
  if resp.status_code != 200:
    break
  
  bs = bs4.BeautifulSoup(resp.text, 'html.parser')
  articles = bs.find_all(name='article')
  for article in articles:
    test_url = article.find('h2', attrs={'itemprop': 'name'}).a['href']
    res = mine_test_page(f'{host}{test_url}')
    results.append(res)
    
  i+=5


UnsupportedPageFromat: 

In [4]:
dfs = []
for val in results:
    df = pd.DataFrame(data=val['ratings'], columns=[x.strip() for x in val['headers']]).assign(url=val['url']).assign(title=val['soup'].title.text).astype({'title': 'str'}).set_index('title').set_index('Név', append=True)
    dfs.append(df)

In [5]:
ratings = pd.concat(dfs);ratings

Krém  \
title                                              Név                                                         
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                               4   
                                                   SicTransit                                            4,5   
                                                   Szabiakos                                             4,5   
                                                   Lorien                                                  4   
                                                   Weinmaker                                               4   
...                                                                                                      ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... \nLuigi/Faema Family örlő és kávéfőző\n             \n2\n   
                                                   \nMonyesz/ Satrap Espressobar (kombi darálóval)\n   \n1\n   
                                                   \nAndrew/Gaggia Pure Espresso, IberItal Challan...  \n1\n   
                                                   \nJobog/Saeco Crema bronzbojlerrel, OPV-vel ell...  \n2\n   
                                                   \nSiD/Olympia Express Club + Anfim Best, Quick ...  \n1\n   

                                                                                                          Íz  \
title                                              Név                                                         
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                               9   
                                                   SicTransit                                            9,5   
                                                   Szabiakos                                             9,5   
                                                   Lorien                                                  8   
                                                   Weinmaker                                               9   
...                                                                                                      ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... \nLuigi/Faema Family örlő és kávéfőző\n             \n2\n   
                                                   \nMonyesz/ Satrap Espressobar (kombi darálóval)\n   \n2\n   
                                                   \nAndrew/Gaggia Pure Espresso, IberItal Challan...  \n2\n   
                                                   \nJobog/Saeco Crema bronzbojlerrel, OPV-vel ell...      2   
                                                   \nSiD/Olympia Express Club + Anfim Best, Quick ...  \n2\n   

                                                                                                        Test  \
title                                              Név                                                         
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                               4   
                                                   SicTransit                                            4,5   
                                                   Szabiakos                                             3,5   
                                                   Lorien                                                  4   
                                                   Weinmaker                                               4   
...                                                                                                      ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... \nLuigi/Faema Family örlő és kávéfőző\n             \n1\n   
                                                   \nMonyesz/ Satrap Espressobar (kombi darálóval)\n   \n0\n   
                                                   \nAndrew/Gaggia Pur

In [6]:
ratings_revamped = ratings.reset_index()

str_columns = ['Név', 'Krém', 'Íz', 'Test', 'Ár', 'Össz:']
numeric_cols = ['Krém', 'Íz', 'Test', 'Ár', 'Össz:']

ratings_revamped[str_columns] = ratings_revamped[str_columns].apply(lambda x: x.str.strip())

ratings_revamped[numeric_cols] = ratings_revamped[numeric_cols].apply(lambda x: x.str.replace(',', '.')).apply(lambda x: x.str.replace('-', ''))

ratings_revamped[numeric_cols] = ratings_revamped[numeric_cols].apply(pd.to_numeric, errors='coerce')

ratings_revamped = ratings_revamped.set_index(['title', 'Név'])

ratings_revamped

Krém  \
title                                              Név                                                        
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                            4.0   
                                                   SicTransit                                           4.5   
                                                   Szabiakos                                            4.5   
                                                   Lorien                                               4.0   
                                                   Weinmaker                                            4.0   
...                                                                                                     ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... Luigi/Faema Family örlő és kávéfőző                  2.0   
                                                   Monyesz/ Satrap Espressobar (kombi darálóval)        1.0   
                                                   Andrew/Gaggia Pure Espresso, IberItal Challange      1.0   
                                                   Jobog/Saeco Crema bronzbojlerrel, OPV-vel ellát...   2.0   
                                                   SiD/Olympia Express Club + Anfim Best, Quick Mi...   1.0   

                                                                                                        Íz  \
title                                              Név                                                       
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                           9.0   
                                                   SicTransit                                          9.5   
                                                   Szabiakos                                           9.5   
                                                   Lorien                                              8.0   
                                                   Weinmaker                                           9.0   
...                                                                                                    ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... Luigi/Faema Family örlő és kávéfőző                 2.0   
                                                   Monyesz/ Satrap Espressobar (kombi darálóval)       2.0   
                                                   Andrew/Gaggia Pure Espresso, IberItal Challange     2.0   
                                                   Jobog/Saeco Crema bronzbojlerrel, OPV-vel ellát...  2.0   
                                                   SiD/Olympia Express Club + Anfim Best, Quick Mi...  2.0   

                                                                                                       Test  \
title                                              Név                                                        
Roastopus Siren Espresso - Honduras kávékóstoló    Rocketeer                                            4.0   
                                                   SicTransit                                           4.5   
                                                   Szabiakos                                            3.5   
                                                   Lorien                                               4.0   
                                                   Weinmaker                                            4.0   
...                                                                                                     ...   
Kávét a közértből? Teszteltük az "olcsó" kategó... Luigi/Faema Family örlő és kávéfőző                  1.0   
                                                   Monyesz/ Satrap Espressobar (kombi darálóval)        0.0   
                                                   Andrew/Gaggia Pure Espresso, IberItal Challange      1.0   
       

In [7]:
ratings_revamped.to_csv('kavekorzo_ratings.csv')

In [8]:
import pandas as pd 
ratings_revamped = pd.read_csv('kavekorzo_ratings.csv')

In [9]:
import plydata as p 

In [10]:
tops = (
  ratings_revamped.rename(columns={'Össz:':'Össz'}).reset_index()
  >> p.group_by('title') 
  >> p.summarise(price='mean(Ár)', crema='mean(Krém)', flavour='mean(Íz)', character='mean(Test)', summa='mean(Össz)', url='first(url)')
#   >> p.query('title.str.contains("Kapu")', engine='python')
  >> p.define(composit='(price + flavour + character)/3')
  >> p.arrange('-summa')
)

In [11]:
from IPython.display import HTML

N = 20
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

s = tops.style.format({'url': make_clickable})

HTML(s.render())

,title,price,crema,flavour,character,summa,url,composit
259,Kapucziner Kávémanufaktúra: Velencei gondolások kedvence - a miénk is,9.800000,4.600000,8.600000,4.200000,27.200000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=588:kapucziner-kavemanufaktura-velencei-gondolasok-kedvence-a-mienk-is&catid=43&Itemid=110,7.533333
58,Go Caffé Dek - egy szuper koffeinmentes,9.600000,4.300000,8.300000,4.300000,26.500000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1723:go-caffe-dek-egy-szuper-koffeinmentes&catid=43&Itemid=110,7.400000
299,Il Moretto Sublime - közvetlen a pörkölőből,8.600000,4.200000,9.000000,4.600000,26.400000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=342:il-moretto-sublime-koezvetlen-a-poerkoelbl&catid=43&Itemid=110,7.400000
22,Kapucziner Karácsonyi Top Blend - tényleg ünnepi kávé,9.000000,4.500000,8.500000,4.400000,26.300000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1847:kapucziner-karacsonyi-top-blend-tenyleg-unnepi-kave&catid=43&Itemid=110,7.300000
79,Red Baggies Raja Toba - villantott a piros naci,9.000000,3.800000,9.000000,4.400000,26.200000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1654:red-baggies-raja-toba-villantott-a-piros-naci&catid=43&Itemid=110,7.466667
49,Roastopus Black Pearl - a polip legyen veled,8.800000,4.200000,9.100000,4.000000,26.100000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1759:roastopus-black-pearl-a-polip-legyen-veled&catid=43&Itemid=110,7.300000
30,Cracker Jack Colombia - az ördög a részletekben lakozik,9.200000,4.000000,9.200000,3.700000,26.100000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1828:cracker-jack-colombia-az-ordog-a-reszletekben-lakozik&catid=43&Itemid=110,7.366667
316,MoKambo Argento kávéteszt,9.400000,4.600000,7.600000,4.400000,26.000000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=198:mokambo-argento-kaveteszt&catid=43&Itemid=110,7.133333
282,Barbera Esperia - amit meg kell kóstolni,7.400000,5.000000,9.200000,4.400000,26.000000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=448:barbera-esperia-amit-meg-kell-kostolni&catid=43&Itemid=110,7.000000
0,Roastopus Siren Espresso - Honduras kávékóstoló,8.600000,4.200000,9.000000,4.000000,25.800000,https://www.kavekorzo.hu/index.php?option=com_content&view=article&id=1910:roastopus-siren-espresso-honduras-kavekostolo&catid=43&Itemid=110,7.200000
